# 使用TensorFlow构建一个简单的线性分类模型

&emsp;&emsp;在这个示例中，将会使用TensorFlow构建一个基本的线性分类模型来对[mnist数据集](http://yann.lecun.com/exdb/mnist/)中的的手写数字进行识别。MNIST数据集是一个经典的分类数据集，它的每个样本是一个28\*28的灰度图，每个图片展示的是手写数字0~9中的一个，如下图示。

![mnist dataset](./img/MNIST.png)

任务目标就是识别数字，即将数字正确的分类，采用模型的一些参数如下：

+ 模型：softmax回归，即逻辑回归在多分类情况下的推广形式，$p(y=i)=\frac{e^{-w_ix}}{\sum_j e^-w_jx}$

+ 特征：图片的灰度值特征

+ 损失函数：log损失函数 $loss = -\sum_i I_{i=lable}logp(y=i)$

In [ ]:
import time

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 参量设置
batch_size = 128 
image_size = 28*28
label_num = 10
step_size = 0.001
max_steps= 10000

# 每个样本特征和标签的占位表示
x = tf.placeholder(tf.float32, [None, image_size])
y_real = tf.placeholder(tf.float32, [None, label_num])

# 构建模型
w = tf.Variable(tf.truncated_normal(shape=(image_size, label_num), stddev=0.1), name="weight")
b = tf.Variable(tf.zeros(shape=label_num), name="bias")
p = tf.nn.softmax(tf.matmul(x, w) + b, dim=1)

# softmax损失
loss = tf.reduce_sum(-y_real*tf.log(p))/batch_size

# 准确度
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(y_real, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 使用梯度下降更新
update = tf.train.GradientDescentOptimizer(step_size).minimize(loss)

# 初始化操作
init_op = tf.global_variables_initializer()

# 迭代更新
with tf.Session() as sess:
    sess.run(init_op)
    start = time.time()
    for i in range(max_steps+1):
        xs, ys = mnist.train.next_batch(batch_size)
        if i % 50 == 0:
            timeCos = time.time() - start
            start = time.time()
            print("Step: {:d}, Test accuracy: {:.3f}, timeCos: {:.1f}".
                  format(i, sess.run(accuracy,feed_dict={x: mnist.test.images,
                                                         y_real: mnist.test.labels}),
                         timeCos* 1000 / 50))
        sess.run(update, feed_dict={x: xs, y_real: ys})
